#Trabajo Final Grupo Nº 2

Rafael Perez	rafaelperezctes@gmail.com
Olinca Ayala	olincayalan88@gmail.com
###@Created on Dec Sat 2 16:57:20 2023
###@author: Grupo Nº 2


# Modulo primers

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [10]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display
import pkg_resources

def instalar_librerias():
    librerias = [
        "import_ipynb",
        "nbconvert",
        "ipywidgets",
        "biopython",
        "pandas",
        "numpy",
        "matplotlib",
        "tqdm",
        "requests",
        "gdown",
        "primer3-py",

    ]

    barra_progreso = widgets.IntProgress(
        value=0,
        min=0,
        max=len(librerias),
        bar_style='info',
        orientation='horizontal'
    )

    etiqueta = widgets.Label(value='Iniciando instalación...')
    display(widgets.VBox([etiqueta, barra_progreso]))

    # Instalación de fasttree y mafft con apt-get (solo ejecutar en Colab)
    subprocess.run(["apt-get", "install", "primer3-py"])

    for libreria in librerias:
        try:
            pkg_resources.require(libreria)
            etiqueta.value = f"{libreria} ya está instalado."
        except pkg_resources.DistributionNotFound:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
        barra_progreso.value += 1

    etiqueta.value = 'Instalación completada.'
# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)

# Enlazar el botón con la función de instalación
boton_instalar.on_click(lambda b: instalar_librerias())

# Mostrar botón
#display(boton_instalar)

#instalar_librerias()


Output()

In [11]:
# @title main

def main():
  instalar_librerias()

#if __name__ == "__main__":
#    main()

In [12]:
#@title funciones extras

# Función para calcular el punto de fusión de un primer
def calculate_tm(primer_sequence):
    import primer3
    from Bio import SeqIO
    if len(primer_sequence) > 60:
        return "Secuencia demasiado larga para cálculo de Tm"
    return primer3.calc_tm(primer_sequence)

# Función para analizar la formación de hairpin
def analyze_hairpin(primer_sequence):
    import primer3
    from Bio import SeqIO
    if len(primer_sequence) > 60:
        return "Secuencia demasiado larga para análisis de hairpin"
    return primer3.calc_hairpin(primer_sequence)

# Función para calcular la homodimerización de un primer
def analyze_homodimer(primer_sequence):
    import primer3
    from Bio import SeqIO
    if len(primer_sequence) > 60:
        return "Secuencia demasiado larga para análisis de homodimer"
    return primer3.calc_homodimer(primer_sequence)

def analyze_complementarity(primer1, primer2):
    import primer3
    from Bio import SeqIO
    return primer3.bindings.calcHomodimer(primer1, primer2)


In [13]:
#@title analyze primers ('tm', 'hairpin','homodimer')

# Función para leer secuencias desde archivos FASTA
def read_fasta_files(file_list):
    import primer3
    from Bio import SeqIO
    sequences = {}
    for file in file_list:
        with open(file, "r") as fasta_file:
            for record in SeqIO.parse(fasta_file, "fasta"):
                sequences[file] = str(record.seq)
    return sequences

# Función para generar una secuencia de primer aleatoria o predeterminada
def generate_default_primer():
    import random
    bases = ['A', 'T', 'C', 'G']
    return ''.join(random.choices(bases, k=20))

# Función para diseñar primers para una secuencia objetivo
def design_primers_for_target_sequence(target_sequence):
    import primer3
    global_args = {
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 40.0,
        'PRIMER_MAX_GC': 60.0,
        'PRIMER_MAX_POLY_X': 5,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 8,
        'PRIMER_MAX_SELF_END': 3,
        'PRIMER_PAIR_MAX_COMPL_ANY': 8,
        'PRIMER_PAIR_MAX_COMPL_END': 3,
        'PRIMER_PRODUCT_SIZE_RANGE': [[100, 300]]
    }
    seq_args = {
        'SEQUENCE_ID': 'test',
        'SEQUENCE_TEMPLATE': target_sequence,
        'SEQUENCE_INCLUDED_REGION': [0, len(target_sequence)]
    }

    return primer3.bindings.design_primers(seq_args, global_args)

# Funciones para análisis de primers
def analyze_primers(primer_sequence):
    import primer3
    return {
        'tm': primer3.bindings.calc_tm(primer_sequence),
        'hairpin': primer3.bindings.calc_hairpin(primer_sequence),
        'homodimer': primer3.bindings.calc_homodimer(primer_sequence)
    }

# Función para procesar archivos FASTA y diseñar primers para cada secuencia objetivo
def imprimir_resultados_primer(file, primer_used, primer_analysis, is_default):
    import primer3
    from Bio import SeqIO
    print(f"Resultados para {file}:")
    print(f"Primer utilizado: {primer_used} {'(Por defecto)' if is_default else ''}")
    print("Análisis de Primer:")
    print(f"  - Temperatura de Fusión (Tm): {primer_analysis['tm']:.2f} °C")

    hairpin = primer_analysis['hairpin']
    print(f"  - Hairpin:")
    print(f"    - Estructura Encontrada: {'Sí' if hairpin.structure_found else 'No'}")
    if hairpin.structure_found:
        print(f"    - Tm: {hairpin.tm:.2f} °C")
        print(f"    - ΔG: {hairpin.dg:.2f} kcal/mol")
        print(f"    - ΔH: {hairpin.dh:.2f} kcal/mol")
        print(f"    - ΔS: {hairpin.ds:.2f} cal/(mol·K)")

    homodimer = primer_analysis['homodimer']
    print(f"  - Homodimer:")
    print(f"    - Estructura Encontrada: {'Sí' if homodimer.structure_found else 'No'}")
    if homodimer.structure_found:
        print(f"    - Tm: {homodimer.tm:.2f} °C")
        print(f"    - ΔG: {homodimer.dg:.2f} kcal/mol")
        print(f"    - ΔH: {homodimer.dh:.2f} kcal/mol")
        print(f"    - ΔS: {homodimer.ds:.2f} cal/(mol·K)")

    print("----------\n")

# Modificación en la función process_and_design_primers para usar la nueva función de impresión
def process_and_design_primers(file_list):
    import primer3
    from Bio import SeqIO
    target_sequences = read_fasta_files(file_list)
    results = {}
    for file, sequence in target_sequences.items():
        print(f"Procesando y diseñando primers para el archivo: {file}")
        primer_design = design_primers_for_target_sequence(sequence)
        primer_used = ""
        is_default = False

        if primer_design['PRIMER_LEFT_NUM_RETURNED'] > 0:
            left_primer_sequence = primer_design['PRIMER_LEFT'][0]['SEQUENCE']
            primer_analysis = analyze_primers(left_primer_sequence)
            primer_used = left_primer_sequence
        else:
            default_primer_sequence = generate_default_primer()
            primer_analysis = analyze_primers(default_primer_sequence)
            primer_used = default_primer_sequence
            is_default = True

        results[file] = {
            'primer_design': primer_design if primer_design['PRIMER_LEFT_NUM_RETURNED'] > 0 else None,
            'primer_analysis': primer_analysis,
            'primer_used': primer_used
        }

        imprimir_resultados_primer(file, primer_used, primer_analysis, is_default)

    return results


# Ejemplo de uso
#all_guides_df = process_and_design_primers(['U12441.2.fasta', 'AY509004.fasta'])


In [14]:
#@title analyze primers ('tm', 'hairpin','homodimer') - graficar

import matplotlib.pyplot as plt

# Mejoras en las funciones de graficación
def plot_tm_comparison(primer_data):
    # Adaptación para la nueva estructura de datos
    primers = list(primer_data.keys())
    tms = [primer_data[primer]['primer_analysis']['tm'] for primer in primers]

    plt.bar(primers, tms)
    plt.xlabel('Primers')
    plt.ylabel('Temperatura de Fusión (Tm)')
    plt.title('Comparación de Tm de Primers')
    plt.show()

def plot_hairpin_homodimer(primer_data):
    # Adaptación para la nueva estructura de datos
    primers = list(primer_data.keys())
    hairpins = [primer_data[primer]['primer_analysis']['hairpin'].structure_found for primer in primers]
    homodimers = [primer_data[primer]['primer_analysis']['homodimer'].structure_found for primer in primers]

    fig, ax = plt.subplots()
    ax.plot(primers, hairpins, label='Hairpins')
    ax.plot(primers, homodimers, label='Homodimers')
    ax.legend()
    plt.xlabel('Primers')
    plt.ylabel('Estructura Encontrada (True/False)')
    plt.title('Análisis de Hairpin y Homodimerización')
    plt.show()

# Llamar a las funciones de graficación con los nuevos datos
#all_guides_df = process_and_design_primers(['U12441.2.fasta', 'AY509004.fasta'])
#plot_tm_comparison(all_guides_df)
#plot_hairpin_homodimer(all_guides_df)


In [17]:
#@title primer con guias

# Función para leer secuencias desde archivos FASTA
def read_fasta_files(file_list):
    from Bio import SeqIO
    sequences = {}
    for file in file_list:
        with open(file, "r") as fasta_file:
            for record in SeqIO.parse(fasta_file, "fasta"):
                sequences[file] = str(record.seq)
    return sequences

# Función para encontrar sitios de guía RNA
def encontrar_sitios_guia_rna(target_sequence, guide_length=20, pam_sequence="NGG"):
    guide_sites = []
    for i in range(len(target_sequence) - guide_length - len(pam_sequence) + 1):
        guide = target_sequence[i:i+guide_length+len(pam_sequence)]
        if guide.endswith(pam_sequence):
            guide_sites.append(guide[:-len(pam_sequence)])  # Secuencia de la guía sin la secuencia PAM
    return guide_sites

# Función para ajustar parámetros y diseñar primers
def ajustar_parametros_y_disenar_primers(seq_template, seq_primer, max_attempts=10):
    import primer3
    temp_anillamiento = 50.0
    min_tm = 55.0
    max_tm = 65.0
    min_gc = 40.0
    max_gc = 60.0

    for intento in range(max_attempts):
        global_args = {
            "PRIMER_TM_FORMULA": 1,
            "PRIMER_SALT_CORRECTIONS": 1,
            "PRIMER_ANNEALING_TEMP": temp_anillamiento,
            "PRIMER_MIN_TM": min_tm,
            "PRIMER_MAX_TM": max_tm,
            "PRIMER_MIN_GC": min_gc,
            "PRIMER_MAX_GC": max_gc,
            "PRIMER_MAX_POLY_X": 5,
            "PRIMER_MAX_SELF_ANY": 8,
            "PRIMER_MAX_SELF_END": 3,
            "PRIMER_PAIR_MAX_COMPL_ANY": 8,
            "PRIMER_PAIR_MAX_COMPL_END": 3,
            "PRIMER_PRODUCT_SIZE_RANGE": [[100, 300]]
        }

        seq_args = {
            'SEQUENCE_ID': 'Guide',
            'SEQUENCE_TEMPLATE': seq_template,
            'SEQUENCE_INCLUDED_REGION': [0, len(seq_template)],
            'SEQUENCE_PRIMER': seq_primer,
        }

        primer_results = primer3.bindings.designPrimers(seq_args, global_args)
        if 'PRIMER_LEFT' in primer_results and primer_results['PRIMER_LEFT']:
            return primer_results

        # Ajustar los parámetros para el próximo intento
        temp_anillamiento -= 2
        min_tm -= 1
        max_tm += 1
        min_gc -= 2
        max_gc += 2

    return None

# Función para diseñar primers para cada guía RNA
def disenar_primers_guia_rna(target_sequence, guide_sites):
    resultados = {}
    for guide in guide_sites:
        print(f"Intentando guía RNA: {guide}")
        primer_results = ajustar_parametros_y_disenar_primers(target_sequence, guide)
        if primer_results:
            resultados[guide] = primer_results
        else:
            print(f"No se pudieron generar primers para la guía RNA: {guide}")

    return resultados

# Función para imprimir los resultados
def imprimir_resultados_guia_rna(resultados):
    for guide, primer_result in resultados.items():
        if primer_result and 'PRIMER_LEFT' in primer_result and primer_result['PRIMER_LEFT']:
            left_primer = primer_result['PRIMER_LEFT'][0]['SEQUENCE']
            right_primer = primer_result['PRIMER_RIGHT'][0]['SEQUENCE']
            print(f"Guía RNA: {guide}, Primer izquierdo: {left_primer}, Primer derecho: {right_primer}")
        else:
            print(f"No se encontraron primers para la guía RNA: {guide}")

# Procesamiento principal
def primers_guias(file_list):
    target_sequences = read_fasta_files(file_list)

    for file, sequence in target_sequences.items():
        print(f"Procesando secuencia de {file}")
        guide_sites = encontrar_sitios_guia_rna(sequence)
        if guide_sites:
            print(f"Sitios de guía RNA encontrados en {file}: {guide_sites}")
            resultados = disenar_primers_guia_rna(sequence, guide_sites)
            imprimir_resultados_guia_rna(resultados)
        else:
            print(f"No se encontraron sitios de guía RNA en {file}")

#ile_list = ['U12441.2.fasta', 'AY509004.fasta']
#primers_guias(file_list)


In [18]:
#@title primer con guias primers

# Función para leer secuencias desde archivos FASTA
def read_fasta_files(file_list):
    from Bio import SeqIO
    sequences = {}
    for file in file_list:
        with open(file, "r") as fasta_file:
            for record in SeqIO.parse(fasta_file, "fasta"):
                sequences[file] = str(record.seq)
    return sequences

# Función para encontrar sitios de guía RNA
def encontrar_sitios_guia_rna(target_sequence, guide_length=20, pam_sequence="NGG"):
    guide_sites = []
    for i in range(len(target_sequence) - guide_length - len(pam_sequence) + 1):
        guide = target_sequence[i:i+guide_length+len(pam_sequence)]
        if guide.endswith(pam_sequence):
            guide_sites.append(guide[:-len(pam_sequence)])  # Secuencia de la guía sin la secuencia PAM
    return guide_sites

# Función para ajustar parámetros y diseñar primers
def ajustar_parametros_y_disenar_primers(seq_template, seq_primer, max_attempts=10):
    import primer3
    temp_anillamiento = 50.0
    min_tm = 55.0
    max_tm = 65.0
    min_gc = 40.0
    max_gc = 60.0

    for intento in range(max_attempts):
        global_args = {
            "PRIMER_TM_FORMULA": 1,
            "PRIMER_SALT_CORRECTIONS": 1,
            "PRIMER_ANNEALING_TEMP": temp_anillamiento,
            "PRIMER_MIN_TM": min_tm,
            "PRIMER_MAX_TM": max_tm,
            "PRIMER_MIN_GC": min_gc,
            "PRIMER_MAX_GC": max_gc,
            "PRIMER_MAX_POLY_X": 5,
            "PRIMER_MAX_SELF_ANY": 8,
            "PRIMER_MAX_SELF_END": 3,
            "PRIMER_PAIR_MAX_COMPL_ANY": 8,
            "PRIMER_PAIR_MAX_COMPL_END": 3,
"PRIMER_PRODUCT_SIZE_RANGE": [[100, 300]]
        }

        seq_args = {
            'SEQUENCE_ID': 'Guide',
            'SEQUENCE_TEMPLATE': seq_template,
            'SEQUENCE_INCLUDED_REGION': [0, len(seq_template)],
            'SEQUENCE_PRIMER': seq_primer,
        }

        primer_results = primer3.bindings.designPrimers(seq_args, global_args)
        if 'PRIMER_LEFT' in primer_results and primer_results['PRIMER_LEFT']:
            return primer_results

        # Ajustar los parámetros para el próximo intento
        temp_anillamiento -= 2
        min_tm -= 1
        max_tm += 1
        min_gc -= 2
        max_gc += 2

    return None

# Función para imprimir los resultados
def imprimir_resultados_guia_rna(resultados):
    for guide, primer_result in resultados.items():
        if primer_result and 'PRIMER_LEFT' in primer_result and primer_result['PRIMER_LEFT']:
            left_primer = primer_result['PRIMER_LEFT'][0]['SEQUENCE']
            right_primer = primer_result['PRIMER_RIGHT'][0]['SEQUENCE']
            print(f"Guía RNA: {guide}, Primer izquierdo: {left_primer}, Primer derecho: {right_primer}")
        else:
            print(f"No se encontraron primers para la guía RNA: {guide}")

# Función modificada para aceptar el dataframe best_guides_df
def primers_guias2(file_list, best_guides_df):
    target_sequences = read_fasta_files(file_list)

    for file, sequence in target_sequences.items():
        print(f"Procesando secuencia de {file}")

        # Filtrar las guías del dataframe para la secuencia actual
        guides_for_sequence = best_guides_df[best_guides_df['File'] == file]

        if not guides_for_sequence.empty:
            resultados = {}
            for _, guide_row in guides_for_sequence.iterrows():
                guide_sequence = guide_row['Guide']
                print(f"Usando guía RNA: {guide_sequence} para {file}")
                primer_results = ajustar_parametros_y_disenar_primers(sequence, guide_sequence)
                if primer_results:
                    resultados[guide_sequence] = primer_results
                else:
                    print(f"No se pudieron generar primers para la guía RNA: {guide_sequence}")
            imprimir_resultados_guia_rna(resultados)
        else:
            print(f"No se encontraron guías RNA específicas en best_guides_df para {file}")

# Ejemplo de uso
# file_list = ['U12441.2.fasta', 'AY509004.fasta']
# best_guides_df = pd.DataFrame(...)  # Suponiendo que best_guides_df se ha cargado adecuadamente
# primers_guias2(file_list, best_guides_df)

